<a href="https://colab.research.google.com/github/AlisonCarv/desafiosJacto/blob/main/desafio3_jacto.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.layers import Input, Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
import os

# 1. Clona o seu repositório do GitHub para o ambiente do Colab
!git clone https://github.com/AlisonCarv/desafiosJacto.git

# 2. Define o caminho correto para o arquivo zip dentro da pasta clonada
caminho_zip = "/content/desafiosJacto/dataset.zip"

# 3. Descompacta o arquivo zip
!unzip -q {caminho_zip}

print("Repositório clonado e dataset descompactado.")

# Verifica se as pastas foram criadas corretamente
print(os.listdir('dataset'))
print(f"imagens saudáveis: {len(os.listdir('dataset/saudavel'))}")
print(f"imagens doentes: {len(os.listdir('dataset/doente'))}")

DIRETORIO_DADOS = 'dataset'
TAMANHO_IMG = (224, 224)
TAMANHO_LOTE = 16 # batch size

# parte 2: preparar os geradores de dados
# eles vão ler as imagens das pastas, prepará-las e entregá-las ao modelo
# validation_split separa 20% dos dados para teste, para vermos se o modelo está generalizando
# data augmentation (rotation, flip) cria variações das imagens para o modelo aprender melhor
gerador_imagens = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

gerador_treino = gerador_imagens.flow_from_directory(
    DIRETORIO_DADOS,
    target_size=TAMANHO_IMG,
    batch_size=TAMANHO_LOTE,
    class_mode='binary',
    subset='training'
)

gerador_validacao = gerador_imagens.flow_from_directory(
    DIRETORIO_DADOS,
    target_size=TAMANHO_IMG,
    batch_size=TAMANHO_LOTE,
    class_mode='binary',
    subset='validation'
)

# parte 3: construir o modelo com transfer learning
# carregamos um modelo famoso (mobilenetv2) que já sabe reconhecer milhares de objetos
modelo_base = MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False, # não queremos a camada final de classificação original
    weights='imagenet'
)
# congelamos o conhecimento do modelo base para que ele não seja alterado no nosso treino
modelo_base.trainable = False

# criamos um novo modelo, adicionando nossas próprias camadas de classificação no final
inputs = Input(shape=(224, 224, 3))
x = modelo_base(inputs, training=False)
x = GlobalAveragePooling2D()(x) # camada para achatar os dados
outputs = Dense(1, activation='sigmoid')(x) # nossa camada de decisão: 0 (doente) ou 1 (saudável)
modelo = Model(inputs, outputs)

modelo.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

print("iniciando o treinamento do modelo...")
# parte 4: treinar o modelo
historico = modelo.fit(
    gerador_treino,
    validation_data=gerador_validacao,
    epochs=10 # 10 passadas completas sobre os dados de treino
)
print("treinamento concluído.")

replace dataset/doente/1.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: dataset descompactado.
Found 32 images belonging to 2 classes.
Found 8 images belonging to 2 classes.
iniciando o treinamento do modelo...
Epoch 1/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 12s 4s/step - accuracy: 0.4583 - loss: 0.7568 - val_accuracy: 0.5000 - val_loss: 0.8458
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 296ms/step - accuracy: 0.6042 - loss: 0.7457 - val_accuracy: 0.3750 - val_loss: 0.8760
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 305ms/step - accuracy: 0.6250 - loss: 0.6083 - val_accuracy: 0.5000 - val_loss: 0.8766
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 290ms/step - accuracy: 0.7292 - loss: 0.6039 - val_accuracy: 0.3750 - val_loss: 0.6864
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 312ms/step - accuracy: 0.9167 - loss: 0.4656 - val_accuracy: 0.2500 - val_loss: 0.8978
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 577ms/step - accuracy: 0.8125 - loss: 0.4666 - val_accuracy: 0.6250 - val_loss: 0.7462
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 

In [ ]:
import numpy as np
from google.colab import files
from tensorflow.keras.utils import load_img, img_to_array

# faz o upload do arquivo de teste
uploaded = files.upload()

# pega o nome do arquivo e o prepara para o modelo
for nome_arquivo in uploaded.keys():
  img = load_img(nome_arquivo, target_size=TAMANHO_IMG)
  img_array = img_to_array(img)
  img_array = tf.expand_dims(img_array, 0) # cria um "batch" com uma única imagem
  img_array = img_array / 255.0

  # faz a previsão
  predicao = modelo.predict(img_array)

  # interpreta o resultado
  # descobre qual classe é qual (o gerador decide isso em ordem alfabética)
  indice_classes = gerador_treino.class_indices
  classes = list(indice_classes.keys())

  print(f"\nO modelo previu com valor: {predicao[0][0]:.4f}")

  if predicao[0][0] > 0.5:
    print(f"Diagnóstico: a folha parece ser '{classes[1]}'") # geralmente 'saudavel'
  else:
    print(f"Diagnóstico: a folha parece ser '{classes[0]}'") # geralmente 'doente'

Saving 2.jpg to 2.jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step

O modelo previu com valor: 0.5359
Diagnóstico: a folha parece ser 'saudavel'
